In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras

In [ ]:
import pandas as pd
import string
import re

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords' , 'punkt')

[nltk_data] Downloading package stopwords to punkt...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def text_preprocess(messages):
  ps = PorterStemmer()
  lemmatizer = WordNetLemmatizer()
  corpus=[]
  for i in range(0 , len(messages)):
    review = re.sub('[^a-zA-Z]' , ' ' , messages[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review);
    corpus.append(review)
  return corpus

In [ ]:
import itertools
import numpy as np
np.random.seed(1337)
from gensim.models import Word2Vec
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization, Bidirectional
from keras.utils import np_utils
from keras.optimizers import RMSprop
from sklearn.utils import shuffle

In [ ]:

min_word_frequency_word2vec = 3
embed_size_word2vec = 100
context_window_word2vec = 5


max_sentence_len = 50
min_sentence_length = 2
batch_size = 32

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving problem_dataset.csv to problem_dataset (2).csv


In [ ]:
import io

In [ ]:
data = pd.read_csv(io.BytesIO(uploaded['problem_dataset.csv']))
data.head()

,ID,stacktrace,original_label
0,0,[{'exception': 'java.lang.ArrayIndexOutOfBound...,arrayindexoutofboundsexception
1,1,[{'exception': 'java.io.UnsupportedEncodingExc...,unsupportedencodingexception
2,2,"[{'exception': 'java.io.IOException', 'message...",ioexception
3,3,[{'exception': 'java.lang.NullPointerException...,nullpointerexception
4,4,[{'exception': 'javax.swing.text.StateInvarian...,stateinvarianterror


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data['stacktrace'] = text_preprocess(data['stacktrace'])

In [ ]:
class_to_predict = 'original_label'

In [ ]:
data.head()

,ID,stacktrace,original_label
0,0,exception java lang arrayindexoutofboundsexcep...,arrayindexoutofboundsexception
1,1,exception java io unsupportedencodingexception...,unsupportedencodingexception
2,2,exception java io ioexception message opt loca...,ioexception
3,3,exception java lang nullpointerexception messa...,nullpointerexception
4,4,exception javax swing text stateinvarianterror...,stateinvarianterror


In [ ]:
  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

In [ ]:
  train_data = [x[0] for x in data_train[['stacktrace']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]
  unique_train_label = list(set(train_labels))


In [ ]:
lable_id = {}
for i,label_ in enumerate(unique_train_label):
  lable_id[i] = label_


In [ ]:
lable_id[-1]

'error'

In [ ]:
len(unique_train_label)

53

In [ ]:
  test_data = [x[0] for x in data_test[['stacktrace']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

In [ ]:
  # Tokenize data
  train_data = [text.split() for text in train_data] # TODO(Vladimir) - try nltk tokenize here
  test_data = [text.split() for text in test_data]
  all_data = train_data + test_data

In [ ]:
  print('Data examples')
  print(all_data[0])

Data examples
['exception', 'java', 'lang', 'arrayindexoutofboundsexception', 'message', 'frame', 'function', 'java', 'util', 'vector', 'elementat', 'file', 'vector', 'java', 'fileline', 'depth', 'function', 'com', 'sun', 'java', 'swing', 'jtabbedpane', 'seticonat', 'file', 'jtabbedpane', 'java', 'fileline', 'depth', 'function', 'com', 'netbeans', 'developer', 'base', 'window', 'multiobjectframe', 'updateicons', 'file', 'multiobjectframe', 'java', 'fileline', 'depth', 'function', 'com', 'netbeans', 'developer', 'base', 'window', 'multiobjectframe', 'access', 'file', 'multiobjectframe', 'java', 'fileline', 'depth', 'function', 'com', 'netbeans', 'developer', 'base', 'window', 'multiobjectframe', 'propertychange', 'file', 'multiobjectframe', 'java', 'fileline', 'depth', 'function', 'com', 'netbeans', 'developer', 'util', 'node', 'node', 'fireownpropertychange', 'file', 'node', 'java', 'fileline', 'depth', 'function', 'com', 'netbeans', 'developer', 'util', 'node', 'node', 'fireiconchange

In [ ]:
  wordvec_model = Word2Vec(all_data, min_count=min_word_frequency_word2vec,
                           vector_size=embed_size_word2vec, window=context_window_word2vec)
  vocabulary = wordvec_model.wv
  vocab_size = len(vocabulary)

In [ ]:
len(vocabulary)

1126

In [ ]:
  X_train = np.empty(shape=[len(train_data), max_sentence_len, embed_size_word2vec],
                     dtype='float32')
  Y_train = np.empty(shape=[len(train_labels), 1], dtype='int32')

In [ ]:
  for j, curr_row in enumerate(train_data):
    if j % 100 == 0:
      print('Building X_train j = ' + str(j))
    sequence_cnt = 0
    for item in curr_row:
      if item in vocabulary:
        X_train[j, sequence_cnt, :] = wordvec_model.wv[item]
        sequence_cnt = sequence_cnt + 1
        if sequence_cnt == max_sentence_len - 1:
          break
    for k in range(sequence_cnt, max_sentence_len):
      X_train[j, k, :] = np.zeros((1, embed_size_word2vec))
    Y_train[j, 0] = unique_train_label.index(train_labels[j])

Building X_train j = 0
Building X_train j = 100
Building X_train j = 200
Building X_train j = 300
Building X_train j = 400


In [ ]:
  X_test = np.empty(shape=[len(test_data), max_sentence_len, embed_size_word2vec],
                    dtype='float32')
  Y_test = np.empty(shape=[len(test_labels), 1], dtype='int32')

In [ ]:
  for j, curr_row in enumerate(test_data):
    if j % 100 == 0:
      print('Building X_test j = ' + str(j))
    sequence_cnt = 0
    for item in curr_row:
      if item in vocabulary:
        X_test[j, sequence_cnt, :] = wordvec_model.wv[item]
        sequence_cnt = sequence_cnt + 1
        if sequence_cnt == max_sentence_len - 1:
          break
    for k in range(sequence_cnt, max_sentence_len):
      X_test[j, k, :] = np.zeros((1, embed_size_word2vec))
    try:
      Y_test[j, 0] = unique_train_label.index(test_labels[j])
    except:
      Y_test[j,0] = -1



Building X_test j = 0


In [ ]:
 y_train = np_utils.to_categorical(Y_train, len(unique_train_label))

In [ ]:
y_train[0].shape

(53,)

In [ ]:
  sequence = Input(shape=(max_sentence_len, embed_size_word2vec), dtype='float32')
  lstm = Bidirectional(LSTM(1024))(sequence)
  after_dp = Dropout(0.5)(lstm)
  output = Dense(len(unique_train_label), activation='softmax')(after_dp)
  model = Model(inputs=sequence, outputs=output)
  rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08)
  model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
hist = model.fit(X_train, y_train,
                   batch_size=batch_size,
                   epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


14/14 [==============================] - 73s 5s/step - loss: 1.5066 - accuracy: 0.5812
Epoch 2/10
14/14 [==============================] - 71s 5s/step - loss: 1.1371 - accuracy: 0.6800
Epoch 3/10
14/14 [==============================] - 71s 5s/step - loss: 1.2011 - accuracy: 0.6588
Epoch 4/10
14/14 [==============================] - 71s 5s/step - loss: 0.9267 - accuracy: 0.7106
Epoch 5/10
14/14 [==============================] - 71s 5s/step - loss: 0.7533 - accuracy: 0.7671
Epoch 6/10
14/14 [==============================] - 70s 5s/step - loss: 0.7041 - accuracy: 0.7882
Epoch 7/10
14/14 [==============================] - 70s 5s/step - loss: 0.6084 - accuracy: 0.8118
Epoch 8/10
14/14 [==============================] - 70s 5s/step - loss: 0.4273 - accuracy: 0.8776
Epoch 9/10
14/14 [==============================] - 70s 5s/step - loss: 0.4574 - accuracy: 0.8659
Epoch 10/10
14/14 [==============================] - 71s 5s/step - loss: 0.4299 - accuracy: 0.8706


In [ ]:
  train_result = hist.history
  print(train_result)

{'loss': [3.762427568435669, 2.4133248329162598, 2.2947840690612793, 1.9484755992889404, 1.8235138654708862, 1.5660163164138794, 1.3975955247879028, 1.1117314100265503, 1.0683798789978027, 0.8862244486808777], 'accuracy': [0.26588234305381775, 0.41647058725357056, 0.4000000059604645, 0.477647066116333, 0.5247058868408203, 0.5905882120132446, 0.6188235282897949, 0.6847058534622192, 0.6705882549285889, 0.7411764860153198]}


In [ ]:
  train_prediction = model.predict(X_train)

14/14 [==============================] - 16s 1s/step


In [ ]:
len(wordvec_model.wv["io"])

100

In [ ]:
  total_train_correct = 0

  pred = []


  for j, ll in enumerate(train_prediction):
    label_predicted = lable_id[np.argmax(ll)]
    pred.append(label_predicted)
    if np.argmax(ll) == Y_train[j]:
      total_train_correct += 1


  print('Train accuracy:', total_train_correct * 1.0 / len(train_prediction))


Train accuracy: 0.9552941176470588


In [ ]:
  test_prediction = model.predict(X_test)


3/3 [==============================] - 3s 869ms/step


In [ ]:
  total_test_correct = 0
  labels = []
  predicted = []
  for j, ll in enumerate(test_prediction):
    predicted.append(np.argmax(ll))
    labels.append(Y_test[j])
    if np.argmax(ll) == Y_test[j]:
      total_test_correct += 1

  print('Test accuracy:', total_test_correct * 1.0 / len(test_prediction))


Test accuracy: 0.6266666666666667
